# Generador de trayectorias (Redes mundo pequeño)
En esta notebook planteo un sistema de N ecuaciones diferenciales de primer orden acopladas mediante el modelo de osciladores Kuramoto que conforman una red mundo pequeño: 

$$ \dot{\theta_i}=\omega_i +\frac{\lambda}{\langle k\rangle}\sum_{j=1}^{N}A_{ij}\sin{(\theta_j -\theta_i)} $$

A su vez, elijo un porcentaje del total de osciladores y los perturbo con una fuerza periodica:

$$ \dot{\theta_i}=\omega_i +\frac{\lambda}{\langle k\rangle}\sum_{j=1}^{N}A_{ij}\sin{(\theta_j -\theta_i)} + b \sin{(\omega_f t - \theta_i)} $$

Donde, para redes mundo pequeño donde cada nodo se conecta hasta su $\kappa$-esimo vecino:

$$\langle k\rangle = 2\kappa$$

La funcion "simulator" recibe como argumentos la cantidad de osciladores N, un array con distintas fuerzas de acoplamiento $\lambda$, el metodo de integracion numerica elegido (En este caso usamos RK4), el tiempo que voy a dejar evolucionando el sistema, la intensidad de la perturbacion externa "b", la frecuencia de la perturbacion periodica externa $\omega_f$, el porcentaje de osciladores que seran perturbados p_per, la lista de vecinos de los osciladores de la red y la probabilidad de reconexion $q$. El codigo simplemente devuelve la resolucion numerica del sistema de N ecuaciones diferenciales acopladas en el tiempo final $t_f$ (Se puede modificar para que devuelva la evolucion temporal completa o en el tiempo que se desee).
Dentro de la funcion simulador, se adapta la distribucion de frecuencias a datos experimentales del NSQ en donde se supuso que las celulas tiene un periodo de $23\mathrm{h}$.

## Paquetes usados

In [1]:
using Plots
using LaTeXStrings
using Distributions
using JLD2
using BenchmarkTools
using Random
using Dates

## Definicion de funciones

In [2]:
function gen_paso_rk4(N)
    k1=zeros(N)
    k2=zeros(N)
    k3=zeros(N)
    k4=zeros(N)    
    function paso_rk4!(f!,dx,x,t,h)
        # calculamos k1
        f!(k1,x,t)
        k1 .*= h
        # calculamos k2
        dx .= x .+ 0.5 .* k1 # Aca usamos dx como variable temporal
        f!(k2,dx,t+0.5*h)
        k2 .*= h
        # calculamos k3
        dx .= x .+ 0.5 .* k2
        f!(k3,dx,t+0.5*h)
        k3 .*= h
        # calculamos k4
        dx .= x .+ k3
        f!(k4,dx,t+h)
        k4 .*= h
        # calculamos dx
        dx .= x .+ (k1 .+ 2 .* k2 .+ 2 .* k3 .+ k4) ./ 6
    end
    return paso_rk4!
end

gen_paso_rk4 (generic function with 1 method)

In [3]:
function integrador_EDO!(method!,f!,vt,vx)
    @assert size(vx)[1]>1
    for s in 2:length(vt)
        h = vt[s]-vt[s-1]
        x = view(vx,:,s-1)
        dx = view(vx,:,s)
        method!(f!,dx,x,vt[s],h)
    end
    return vx
end

integrador_EDO! (generic function with 1 method)

In [4]:
# Este integrador no devuelve la trayectoria completa, solo devuelve los puntos finales. Esto deberia agilizar el
# analisis de datos posterior.

function integrador_EDO_rapido!(method!,f!,vt,vx)
    @assert size(vx)[1]>1
    i1,i2=1,2
    for s in 2:length(vt)
        h = vt[s]-vt[s-1]
        x = view(vx,:,i1)
        dx = view(vx,:,i2)
        method!(f!,dx,x,vt[s],h)
        i1,i2=i2,i1         
    end
    return view(vx,:,i1)
end

integrador_EDO_rapido! (generic function with 1 method)

## Generador de una red regular

In [5]:
# Con la siguiente funcion genero la lista de vecinos para una red regular
#
#      N: Cantidad de nodos en la red
#      k: Hasta que vecino se conecta cada nodo (Cada nodo tendra 2k conexiones)
#
# IMPORTANTE:
#
#   Se puede elegir devolver dos variables: edgelist y edgelist_2
#
#      edgelist: Contiene las conexiones como un array de tuplas: [(1,2),(2,1)] 
#                (Mas util para construir una SW)
#      edgelist_2: Contiene las conexiones como una matriz : [1 2, 2 1] 
#                (Este es el formato que necesita el codigo del simulador de Kuramoto)

function gen_edgelist_regular(N, k)
    edgelist = []
    for i in 1:N
        for j in 1:(k+1)
            vecino = mod(i + j - 2, N) + 1 # Calculo el indice del vecino
            if i != vecino  # Evitar conexiones a sí mismo
                push!(edgelist, (i,vecino))
                push!(edgelist, (vecino,i))
            end
        end
    end
    sort!(edgelist)
    edgelist_2 = zeros(Int,N*2*k,2)
    for i in 1:length(edgelist)
        edgelist_2[i,1]=edgelist[i][1]
        edgelist_2[i,2]=edgelist[i][2]
    end
    return edgelist
end

gen_edgelist_regular (generic function with 1 method)

## Generador de red mundo pequeño

In [6]:
# Con la siguiente funcion genero la lista de vecinos para una red Small World
#
#      N: Cantidad de nodos en la red
#      k: Hasta que vecino se conecta cada nodo inicialmente (Cada nodo tendra 2k conexiones)
#      p_rewiring: probabilidad de que un link se reconecte 
#
# IMPORTANTE:
#
#   Se puede elegir devolver dos variables: edgelist y edgelist_2
#
#      edgelist_sw: Contiene las conexiones como un array de tuplas: [(1,2),(2,1)] 
#                (Mas util para construir una SW)
#      edgelist_sw_2: Contiene las conexiones como una matriz : [1 2, 2 1] 
#                (Este es el formato que necesita el codigo del simulador de Kuramoto)
#
function gen_edgelist_small_world(N,k,p_rewiring)
    # Genero la lista de vecinos de una red regular
    regular_original = gen_edgelist_regular(N,k)
    edgelist_sw = copy(regular_original)
    # Genero una lista vacia donde van a ir las conexiones de la red SW
    for (i,j) in edgelist_sw   # Recorro todos los links (i,j) de la red
        
        # Sorteo la probabilidad para reconectar el link
        if rand() < p_rewiring
            # ELijo un nodo al azar para conectarlo al nodo i-esimo
            nodo_random = rand(1:N)
            # Evito que el nodo se conecte con sigo mismo, fuerzo a que el nodo al azar no sea el mismo de antes y
            # y que el nuevo link no sea un link existente
            while (nodo_random == i || nodo_random == j) & ((i,nodo_random) in edgelist_sw) & ((nodo_random, j) in edgelist_sw)
                nodo_random = rand(1:N)
            end
            # Agrego el nuevo link a la lista y elimino el viejo
            replace!(edgelist_sw, (i,j) =>(i,nodo_random), (j,i) =>(nodo_random,i))
        end
    end
    # Ordeno la lista de conexiones
    sort!(edgelist_sw)
    edgelist_sw_2 = zeros(Int,N*2*k,2)
    for i in 1:length(edgelist_sw)
        edgelist_sw_2[i,1]=edgelist_sw[i][1]
        edgelist_sw_2[i,2]=edgelist_sw[i][2]
    end
    return edgelist_sw_2
end

gen_edgelist_small_world (generic function with 1 method)

## Simulador

In [9]:
# Para simular genero una funcion que lo haga todo. Le paso el integrador que voy a usar, la cantidad de osciladores "N", una lista con valores de λ y vector "vt" que tiene
# el rango de tiempos en el cual voy a trabajar.
#
# Inputs:
#       integrador!: El integrador que se usara para el sistema de ecuaciones
#       N: Cantidad de osciladores
#       array_λ: Valores de la fuerza de acoplamiento a utilizar
#       vt: El tiempo que dejo evolucionar al sistema
#       p: Probabilidad de conexion de la red
#       edgelist: Lista de vecinos de la red 
#       b: Intensidad de la fuerza periodica pertubativa
#       ω_f: Frecuencia de la fuerza periodica perturbativa
#       p_pert: Porcentaje del total de osciladores a ser perturbados
 
# Return: 
#         Esta celda devuelve un array 1x7: 
#               [b: Intensidad de la perturbacion,
#                N: Cantidad de osciladores, 
#                λ: Intensidad de acoplamiento,
#                vt: Vector tiempo,
#                vθ: Trayectorias(N x len(vt)),
#                indices_perturbar: array con los osciladores que fueron perturbados,
#                ω: Frecuencias intrinsecas de los osciladores]

function simulation(integrador!,N,array_λ,vt,p_rew,edgelist,b,ω_f,p_pert)
    paso_rk4! = gen_paso_rk4(N)
    vθ = zeros(N,length(vt))
    ω = zeros(N)

    # Defino los osciladores que voy a perturbar
    N_per = Int(round(N*p_pert)) # Cantidad de osciladores perturbados

    muestras = []
    for λ in array_λ
        
        # Sorteo los indices de los osciladores que voy a perturbar
        indices_perturbar = sample(1:1:N, N_per, replace=false) 
        
        function f_kuramoto!(dθ,θ,t) # Defino los osciladores
            i_anterior = 1
            suma = 0
            for e in 1:size(edgelist)[1]
                i=edgelist[e,1]
                j=edgelist[e,2]
                if i-i_anterior > 1 # Chequear si salteamos algun indice (Indicador de oscilador/es desacoplado/s)
                    indice = i-1 # Agrego los osciladores desacoplados a la lista
                    while indice > i_anterior
                        dθ[indice] = ω[indice]
                        indice = indice -1
                    end
                end
                if i_anterior != i
                    dθ[i_anterior] = ω[i_anterior]+(λ/(2*k))*suma
                    i_anterior = i
                    suma = 0
                end
                suma += sin(θ[j]-θ[i])
                if e == size(edgelist)[1]
                    dθ[i_anterior]= ω[i_anterior] + (λ/(2*k))*suma
                    index = i
                    while index < N
                        dθ[index+1] = ω[index+1]
                        index = index + 1
                    end
                end
            end
            
            # Contingencia para el caso en que no haya NINGUN nodo conectado
            if size(edgelist)[1] == 0 
                for j in 1:N
                    dθ[j] = ω[j]
                end
            end

            # # Ahora perturbo con la fuerza periodica externa
            for k in 1:length(indices_perturbar)
                index = indices_perturbar[k]
                dθ[index] += b*sin(ω_f*t - θ[index])
            end
            
        end
        # Condiciones iniciales
        vθ[:,1] .= rand(Truncated(Normal(π,2), 0, 2π), N)
        # Frecuencias adaptadas a datos experimentales del NSQ
        μ = 0.2739
        σ = 0.0122
        ω = rand(Normal(μ,σ), N)
        # Integrador
        muestra = integrador!(paso_rk4!,f_kuramoto!,vt,vθ)
        push!(muestras,[b,N,λ,vt,copy(muestra[:,end]), indices_perturbar, ω])
    end
    return muestras
end

simulation (generic function with 2 methods)

In [10]:
n_muestras = 5 # Cantidad de muestras por red
n_redes = 1 # Cantidad de redes
trayectorias = Array{Any}(undef, n_muestras, n_redes)

N = 6 # Tamaño
κ = 3 # Hasta que vecino se conecta cada nodo
q = 0.0059 # Probabilidad de reconexion

b = 0.1 # Intensidad de la perturbacion
ω_z = 0.262 # Frecuencia del zeitgeber
p_per = 0.1 # Porcentaje de osciladores perturbados

λ = 0:0.002:0.8 # λs recorridos
t = 0:8:75 # Tiempo de integracion

@time for m in 1:n_redes
    edgelist = gen_edgelist_small_world(N,κ,q)
    task = Threads.@spawn begin
        Threads.@threads for n in 1:n_muestras
            trayectorias[n,m] = simulation(integrador_EDO!, N, λ, t, q, edgelist, b, ω_z, p_per)
        end
    end
    wait(task)
end

  8.053681 seconds (10.60 M allocations: 464.485 MiB, 3.70% gc time, 91.12% compilation time)


In [15]:
# Aca cargo o genero un diccionario sobre el cual almacenar los datos
data = Dict()
try
    # We load the data dictionary if it exists
    global data = load("SW Perturbado "*string(p_per*100)*"% b="*string(b)*" N="*string(N)*".jld2")
catch
    # Otherwise we create the dictionary
    println("Estas usando un diccionario vacio")
end

Dict{String, Any} with 1 entry:
  "Red SW κ=3 q=0.0059 b=0… => Any[Any[Any[0.1, 6, 0.0, 0:8:72, [23.6027, 23.07…

In [13]:
# Almaceno los resultados en el diccionario usando la fecha actual como llave del mismo
data["Red SW κ="*string(κ)*" q="*string(q)*" b="*string(b)*" N="*string(N)*" "*string(p_per*100)*"% perturbado - "*Dates.format(Dates.now(),"yyyy-mm-dd-HH-MM-SS")]=trayectorias;

In [14]:
# Guardo un archivo con los datos para su posterior analisis.
save("SW Perturbado "*string(p_per*100)*"% b="*string(b)*" N="*string(N)*".jld2",data)